In [ ]:
class CelebADataset(Dataset):
    def __init__(self, img_dir, attr_path, partition_path,
                 target_attr=None, mode='train', NoiseTransform=None):
        """
        CelebA veri seti için Dataset sınıfı.

        Args:
            img_dir (str): Görüntü klasörü (img_align_celeba).
            attr_path (str): list_attr_celeba.csv dosya yolu.
            partition_path (str): list_eval_partition.csv dosya yolu.
            target_attr (list or str, optional): Etiket olarak kullanılacak sütun(lar).
            mode (str): 'train', 'val' veya 'test'.
            NoiseTransform (callable, optional): Ek bozulma veya dönüşüm fonksiyonu.
        """
        assert mode in ['train', 'val', 'test'], "mode 'train', 'val' ya da 'test' olmalıdır."
        self.img_dir = img_dir
        self.mode = mode
        self.NoiseTransform = NoiseTransform

        # Dönüşümler
        self.transform = T.Compose([
            T.CenterCrop(178),
            T.Resize((64, 64)),
            T.ToTensor()
        ])

        # Etiketleri oku
        attr_df = pd.read_csv(attr_path)
        attr_df.set_index("image_id", inplace=True)
        attr_df = (attr_df + 1) // 2  # -1 → 0

        # Partition dosyası
        partition_df = pd.read_csv(partition_path)
        partition_map = {'train': 0, 'val': 1, 'test': 2}
        partition_value = partition_map[mode]
        partition_df = partition_df[partition_df['partition'] == partition_value]

        # İlgili örnekleri seç
        attr_df = attr_df.loc[partition_df['image_id']]

        self.image_ids = attr_df.index.tolist()
        self.labels = attr_df[target_attr].astype('float32').values if target_attr else None

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_name = self.image_ids[idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert("RGB")

        # Temel dönüşümler (crop → resize → tensor)
        image = self.transform(image)

        # NoiseTransform uygulanmış versiyonu
        if self.NoiseTransform:
            image_ = self.NoiseTransform(image).to(torch.float32)
            if self.labels is not None:
                label = torch.tensor(self.labels[idx])
                return image, image_, label
            return image, image_

        # Etiketsiz veya gürültüsüz çıkış
        if self.labels is not None:
            label = torch.tensor(self.labels[idx])
            return image, label

        return image

In [ ]:
"""
class CelebADataset(Dataset):
    def __init__(self, img_dir, attr_path, partition_path, target_attr=None, transform=True, mode='train'):
        self.img_dir = img_dir
        self.mode = mode

        self.transform = transforms.Compose([
            transforms.CenterCrop(178),
            transforms.Resize(64),
            transforms.ToTensor()
        ])

        # Etiketler
        attr_df = pd.read_csv(attr_path)
        attr_df.set_index("image_id", inplace=True)
        attr_df = (attr_df + 1) // 2  # -1 → 0

        # Partition
        partition_df = pd.read_csv(partition_path)
        partition_map = {'train': 0, 'val': 1, 'test': 2}
        partition_value = partition_map[mode]
        partition_df = partition_df[partition_df['partition'] == partition_value]

        # Eşleşen örnekleri al
        attr_df = attr_df.loc[partition_df['image_id']]

        self.image_ids = attr_df.index.tolist()
        self.labels = attr_df[target_attr].astype('float32').values if target_attr else None

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_name = self.image_ids[idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)

        if self.labels is not None:
            label = torch.tensor(self.labels[idx])
            return image, label

        return image
"""